<a href="https://colab.research.google.com/github/moin-84/Quantum-Chemistry-Simulation/blob/main/ms-H2-101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip3 install qiskit qiskit-nature pyscf

In [16]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.problems import ElectronicStructureProblem
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit.circuit.library import TwoLocal
from qiskit.primitives import Estimator
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms import VQE

In [17]:
driver = PySCFDriver(atom="H 0 0 0; H 0 0 0.735", basis="sto-3g")
driver

In [18]:
es_problem = driver.run()
es_problem

In [19]:
active_space = ActiveSpaceTransformer(2, 2)
active_space

In [20]:
transformed_problem = active_space.transform(es_problem)
transformed_problem

In [21]:
mapper = JordanWignerMapper()
mapper

In [22]:
qubits = mapper.map(transformed_problem.hamiltonian.second_q_op())
qubits

SparsePauliOp(['IIII', 'IIIZ', 'IIZI', 'IZII', 'ZIII', 'IIZZ', 'IZIZ', 'ZIIZ', 'YYYY', 'XXYY', 'YYXX', 'XXXX', 'IZZI', 'ZIZI', 'ZZII'],
              coeffs=[-0.81054798+0.j,  0.17218393+0.j, -0.22575349+0.j,  0.17218393+0.j,
 -0.22575349+0.j,  0.12091263+0.j,  0.16892754+0.j,  0.16614543+0.j,
  0.0452328 +0.j,  0.0452328 +0.j,  0.0452328 +0.j,  0.0452328 +0.j,
  0.16614543+0.j,  0.17464343+0.j,  0.12091263+0.j])

In [23]:
ansatz = TwoLocal(rotation_blocks="ry", entanglement_blocks="cz", reps=2)
ansatz

In [24]:
ansatz = TwoLocal(rotation_blocks=["ry", "rx", "rz"], entanglement_blocks="cz", reps=4)
ansatz

In [25]:
estimator = Estimator()
optimizer = COBYLA(maxiter=100)

<ipython-input-25-9e721a099a57>:1: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()


In [26]:
vqe = VQE(estimator, ansatz, optimizer)
vqe

In [27]:
result = vqe.compute_minimum_eigenvalue(qubits)
result

In [28]:
print(f"Ground State Energy of H2: {result.eigenvalue.real:.6f} Hartree")

Ground State Energy of H2: -1.288531 Hartree
